In [1]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]
print(all_X)
print(all_y)
# print(type(all_X))



         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [2]:
from sklearn.ensemble import RandomForestClassifier
#アップサンプリング前のクラス1のデータ数とクラス0のデータ数の出力
print("Number of class 1 samples before:", all_y[all_y == 1].shape[0])
print("Number of class 0 samples before:", all_y[all_y == 0].shape[0])
# print(all_X[all_y == 1])
# print(all_X[all_y == 0])

#クラスの1のサンプルの個数がクラス0と同じになるまで新しいサンプルを復元抽出
X_upsampled, y_upsampled = resample(all_X[all_y == 1],
                                    all_y[all_y == 1],
                                    replace = True,
                                    n_samples = all_X[all_y == 0].shape[0],
                                    random_state = 123)
# アップサンプリング後のクラス１のデータ数とデータの中身の出力
print("Number of class 1 samples after:", X_upsampled.shape[0])
print(X_upsampled)
# 元のクラス０のサンプルにアップサンプリングしたクラス０のサブセットを結合
X_bal = np.vstack((all_X[all_y == 0], X_upsampled))
y_bal = np.hstack((all_y[all_y == 0], y_upsampled))
print(X_bal.shape, y_bal.shape)

X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.3, random_state=0, stratify=y_bal )

# データのテキストへの書き込み
#path = 'upsampled_claim_toiawase_data.txt'
# with open(path, mode='w') as f:
#     f.write(str(X_upsampled))

# with open(path) as f:
#     print(f.read())

Number of class 1 samples before: 1416
Number of class 0 samples before: 8397
Number of class 1 samples after: 8397
[[0.2  2.   0.2  ... 0.   0.   0.56]
 [0.1  1.   0.1  ... 0.   0.   0.93]
 [2.1  4.   2.3  ... 0.2  0.2  0.63]
 ...
 [1.7  3.   1.7  ... 0.   0.   0.75]
 [1.3  2.   1.3  ... 0.   0.   0.92]
 [0.8  2.   2.1  ... 0.33 0.33 0.53]]
(16794, 30) (16794,)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

for i in [2,3,4,5,6,7,8,9,10]:
    X_bal = X_bal[:16794,:i]
    print("変数の数:%s" %i)
    y_bal = y_bal
    scoring = ["f1", "accuracy", "precision", "recall"]
    for c in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]:
        for g in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]:
            print("C:%s      gamma:%s" %(c,g))
            pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
            scores = cross_validate(pipe, X_bal, y_bal, scoring=scoring, cv=10)
            for key,value in scores.items():
                print("{}:{:.4f}+/-{:.4f}".format(key, value.mean(), value.std()))
            print("\n")

変数の数:2
C:1e-06      gamma:1e-06
fit_time:8.4923+/-0.0816
score_time:2.5680+/-0.0360
test_f1:0.0125+/-0.0047
train_f1:0.0125+/-0.0005
test_accuracy:0.5032+/-0.0012
train_accuracy:0.5032+/-0.0001
test_precision:1.0000+/-0.0000
train_precision:1.0000+/-0.0000
test_recall:0.0063+/-0.0024
train_recall:0.0063+/-0.0003


C:1e-06      gamma:1e-05
fit_time:11.1149+/-5.2516
score_time:3.7770+/-2.3424
test_f1:0.0125+/-0.0047
train_f1:0.0125+/-0.0005
test_accuracy:0.5032+/-0.0012
train_accuracy:0.5032+/-0.0001
test_precision:1.0000+/-0.0000
train_precision:1.0000+/-0.0000
test_recall:0.0063+/-0.0024
train_recall:0.0063+/-0.0003


C:1e-06      gamma:0.0001
fit_time:21.5767+/-0.1372
score_time:8.4300+/-0.0700
test_f1:0.0125+/-0.0047
train_f1:0.0125+/-0.0005
test_accuracy:0.5032+/-0.0012
train_accuracy:0.5032+/-0.0001
test_precision:1.0000+/-0.0000
train_precision:1.0000+/-0.0000
test_recall:0.0063+/-0.0024
train_recall:0.0063+/-0.0003


C:1e-06      gamma:0.001
fit_time:21.5079+/-0.0245
score_time:8

fit_time:21.3757+/-0.0204
score_time:8.3587+/-0.0075
test_f1:0.0323+/-0.0079
train_f1:0.0321+/-0.0006
test_accuracy:0.5082+/-0.0020
train_accuracy:0.5081+/-0.0002
test_precision:0.9941+/-0.0176
train_precision:0.9928+/-0.0024
test_recall:0.0164+/-0.0041
train_recall:0.0163+/-0.0003


C:0.0001      gamma:0.01
fit_time:21.3652+/-0.0537
score_time:8.3659+/-0.0098
test_f1:0.2281+/-0.0233
train_f1:0.2282+/-0.0017
test_accuracy:0.5628+/-0.0077
train_accuracy:0.5627+/-0.0006
test_precision:0.9700+/-0.0138
train_precision:0.9706+/-0.0019
test_recall:0.1295+/-0.0149
train_recall:0.1293+/-0.0011


C:0.0001      gamma:0.1
fit_time:21.4360+/-0.0175
score_time:8.3636+/-0.0077
test_f1:0.5235+/-0.0178
train_f1:0.5241+/-0.0021
test_accuracy:0.6664+/-0.0082
train_accuracy:0.6667+/-0.0011
test_precision:0.9160+/-0.0268
train_precision:0.9159+/-0.0020
test_recall:0.3670+/-0.0192
train_recall:0.3670+/-0.0019


C:0.0001      gamma:1.0
fit_time:21.5371+/-0.0166
score_time:8.3653+/-0.0075
test_f1:0.6899+/-0.

fit_time:15.6779+/-0.3280
score_time:5.0708+/-0.0608
test_f1:0.6958+/-0.0198
train_f1:0.6963+/-0.0022
test_accuracy:0.7294+/-0.0214
train_accuracy:0.7299+/-0.0026
test_precision:0.7968+/-0.0406
train_precision:0.7952+/-0.0061
test_recall:0.6186+/-0.0196
train_recall:0.6193+/-0.0034


C:0.01      gamma:10.0
fit_time:15.3792+/-0.2703
score_time:5.1345+/-0.0667
test_f1:0.7311+/-0.0238
train_f1:0.7318+/-0.0031
test_accuracy:0.7343+/-0.0299
train_accuracy:0.7355+/-0.0034
test_precision:0.7431+/-0.0415
train_precision:0.7423+/-0.0058
test_recall:0.7207+/-0.0181
train_recall:0.7216+/-0.0061


C:0.01      gamma:100.0
fit_time:14.4760+/-0.3496
score_time:5.4589+/-0.0897
test_f1:0.7431+/-0.0251
train_f1:0.7427+/-0.0028
test_accuracy:0.7331+/-0.0314
train_accuracy:0.7331+/-0.0035
test_precision:0.7185+/-0.0371
train_precision:0.7169+/-0.0043
test_recall:0.7705+/-0.0208
train_recall:0.7705+/-0.0023


C:0.01      gamma:1000.0
fit_time:15.3311+/-0.5012
score_time:5.9966+/-0.2329
test_f1:0.7431+/-0.0

fit_time:15.2488+/-1.8918
score_time:6.1961+/-0.7265
test_f1:0.7275+/-0.0205
train_f1:0.7319+/-0.0034
test_accuracy:0.7414+/-0.0247
train_accuracy:0.7467+/-0.0026
test_precision:0.7721+/-0.0405
train_precision:0.7772+/-0.0030
test_recall:0.6891+/-0.0206
train_recall:0.6915+/-0.0056


C:1.0      gamma:10000.0
fit_time:13.3095+/-1.1934
score_time:5.4611+/-0.7736
test_f1:0.7275+/-0.0205
train_f1:0.7319+/-0.0034
test_accuracy:0.7414+/-0.0247
train_accuracy:0.7467+/-0.0026
test_precision:0.7721+/-0.0405
train_precision:0.7772+/-0.0030
test_recall:0.6891+/-0.0206
train_recall:0.6915+/-0.0056


C:1.0      gamma:100000.0
fit_time:13.6187+/-1.1440
score_time:5.0471+/-0.4346
test_f1:0.7275+/-0.0205
train_f1:0.7319+/-0.0034
test_accuracy:0.7414+/-0.0247
train_accuracy:0.7467+/-0.0026
test_precision:0.7721+/-0.0405
train_precision:0.7772+/-0.0030
test_recall:0.6891+/-0.0206
train_recall:0.6915+/-0.0056


C:10.0      gamma:1e-06
fit_time:30.1161+/-2.6520
score_time:11.7067+/-1.5391
test_f1:0.0607+/

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


# 標準化、SVCのパイプラインの生成
pipe_svm = make_pipeline(StandardScaler(), SVC( random_state=2 ))
# グリッドサーチ
param_range = [0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0]
params = [{"svc__C":param_range, "svc__kernel":["linear"]},{"svc__C":param_range, "svc__gamma":param_range, "svc__kernel":["rbf"]}]
gs = GridSearchCV(estimator = pipe_svm, param_grid = params, scoring = "accuracy", cv = 3, n_jobs = -1)
# グリッドサーチモデルを適合
gs = gs.fit(X_train, y_train)
# グリッドサーチのスコアの出力
print("Gridsearch_CV_score: %.3f" % gs.best_score_)
# その時のパラメータの組み合わせを出力
print("Gridsearch_best_param: %s" % gs.best_params_)



# ベストとなったパラメータのモデルを用いてテストデータでの予測を実施
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print("Test accuracy : %.3f" % clf.score(X_test, y_test))







from sklearn.metrics import confusion_matrix
#pipe_best_svmのテストにおける予測ラベルの出力
print(pipe_best_svm.predict(X_test))
pre_test = pipe_best_svm.predict(X_test)
confusion = confusion_matrix(y_test, pre_test, labels=[1,0])
print(confusion)
print(confusion[0,0])
tp = confusion[0,0]
fn = confusion[0,1]
fp = confusion[1,0]
tn = confusion[1,1] 